In [5]:
import numpy as np
import scipy.sparse as sp

from src.utils import train_and_test_model, write_submission
from src.recommender_model import RecommenderModel

## Building the Top Recommender model
Seen items are removed from the recommendations

In [6]:
class TopPopRecommender(RecommenderModel):
	def __init__(self):
		super(TopPopRecommender, self).__init__()
		self.urm_train: sp.csr_matrix | None = None
		self.items_popularity_ranking: np.ndarray | None = None

	def fit(self, urm_train: sp.csr_matrix, icm: sp.csr_matrix) -> None:
		self.urm_train = urm_train
		items_popularity = np.ediff1d(urm_train.tocsc().indptr)
		self.items_popularity_ranking = np.argsort(-items_popularity)

	def recommend(self, user_id: int, at: int = 10) -> np.ndarray:
		seen_items = self.urm_train.indices[self.urm_train.indptr[user_id]:self.urm_train.indptr[user_id + 1]]
		unseen_items_mask = np.isin(self.items_popularity_ranking, seen_items, assume_unique=True, invert=True)

		return self.items_popularity_ranking[unseen_items_mask][:at]

## Testing the model

In [7]:
top_pop = train_and_test_model(TopPopRecommender())


KeyboardInterrupt



In [4]:
write_submission(top_pop, "top_pop_recommendation.csv")